In [ ]:
import rasterio
import numpy as np
from sklearn.cluster import KMeans
from skimage.morphology import remove_small_objects, remove_small_holes
import matplotlib.pyplot as plt

In [ ]:
image_path = "../data/raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip.tif"
output_path = "../data/raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip_Hybrid_mask.tif"

In [ ]:
with rasterio.open(image_path) as src:
    img = src.read()
    profile = src.profile
    print(f"Loaded {img.shape[0]} bands, shape = {img.shape[1:]}")
green = img[2].astype(float)
red = img[4].astype(float)
nir = img[6].astype(float)
H, W = red.shape
ndvi = (nir - red) / (nir + red + 1e-6)
brightness = (red + green) / 2.0
brightness_norm = (brightness - brightness.min()) / (brightness.max() - brightness.min())
mask_ndvi = np.where((ndvi < 0.25) & (brightness_norm > 0.5), 1, 0).astype(np.uint8)
bands = np.stack([green, red, nir], axis=-1)  # (H, W, 3)
H, W, C = bands.shape
print(f"Bands stacked: {C} channels, image size {H}x{W}")
bands = np.stack([green, red, nir], axis=-1)
flat = bands.reshape(-1, 3)
flat_norm = (flat - flat.min(axis=0)) / (flat.max(axis=0) - flat.min(axis=0) + 1e-6)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = kmeans.fit_predict(flat_norm)
labels_img = labels.reshape(H, W)
mean_ndvi_per_cluster = [
    np.mean(ndvi[labels_img == i]) if np.sum(labels_img == i) > 0 else np.inf
    for i in range(3)
]
landslide_cluster = np.argmin(mean_ndvi_per_cluster)
mask_kmeans = (labels_img == landslide_cluster).astype(np.uint8)
mask_intersection = ((mask_ndvi == 1) & (mask_kmeans == 1)).astype(np.uint8)
mask_union = ((mask_ndvi == 1) | (mask_kmeans == 1)).astype(np.uint8)
mask_clean = remove_small_objects(mask_union.astype(bool), min_size=100)
mask_clean = remove_small_holes(mask_clean, area_threshold=200)
mask_clean = mask_clean.astype(np.uint8)

In [ ]:
profile.update(dtype=rasterio.uint8, count=1)
with rasterio.open(output_path, "w", **profile) as dst:
    dst.write(mask_clean, 1)
print(f" Hybrid landslide mask saved to:\n{output_path}")